In [34]:
import numpy as np
from bond_pricing import bond_price, bond_coupon_periods, bond_yield
from scipy.optimize import newton

### Original Issue Discount Bonds

The key tax aspect of an OID bond is that the yearly movement along the constant‐yield price trajectory is the reported ordinary interest income to the investor and interest expense to the issuer.

An investor who buys and holds to maturity an OID bond will not have a capital gain because the discount will be taxed as ordinary income over the lifetime of the bond. The problem is that this is “phantom” income in that there is a tax liability each year despite the absence of a cash receipt to pay the tax. That creates a market segmentation effect—most zero‐coupon bonds such as Treasury STRIPS are owned by defined benefit pension funds or by individuals in their retirement savings accounts, for example, 401(k)s. 

The other significant aspect to OID taxation is that capital gains and losses are measured from the constant‐yield price trajectory if the bond is sold prior to maturity.

These tax rules for OID bonds match the economic fundamentals in that interest income is the price change caused purely by the passage of time. A capital gain or loss is the price movement caused by the change in value, meaning a change in the bond yield.

The investor’s after‐tax rate of return depends on the ordinary tax rates that prevail in each year and on the capital gains rate if the zero‐coupon bond is sold prior to maturity.

The investor *credits* interest income and *debits* the bond investment by the interest income generated through passage of time. Those entries raise the *carrying book value* (and the basis for taxation) of the zero‐coupon bond. The capital gain represents the profit from selling at a price above the carrying book value amount.

In [52]:
n_mat = 10
buy_price = 60
sell_price = 68
face = 100
n_sell = 2
per = 2 #semiannual compounding
ytm = ((face/buy_price)**(1/(n_mat*per)) - 1) * per
income_tax = 0.25
capital_tax = 0.15

horizon_yield = ((sell_price/buy_price)**(1/(n_sell*per)) - 1)*per
print(f'Holding Period Return {horizon_yield:.3%} (s.a.), {n_sell} years')

constant_yield_price = [face / ((1 + ytm/per)**((n_mat-i)*per)) for i in range(n_mat)]
taxable_int_income = [constant_yield_price[i] - constant_yield_price[i-1] for i in range(1, n_mat)]
income_tax = [inc * (income_tax) for inc in taxable_int_income]

for i in range(n_sell):
    print(f'Constant Yield Price ({ytm:.3%}) = {constant_yield_price[i+1]:0.3f}, year {i+1}')
    print(f'Taxable Interest Income {taxable_int_income[i]:0.3f}', f'Income Tax = {income_tax[i]:.3f}')

taxable_capital_gain = sell_price - constant_yield_price[2] 
capital_gain_tax = taxable_capital_gain * capital_tax
print(f'Capital Gains Tax = {capital_gain_tax:0.3f} on Capital Gain = {taxable_capital_gain:0.3f}')


at_cf = np.array([-income_tax[0], sell_price-capital_gain_tax-income_tax[1]])
period = np.arange(1, n_sell+1)

def aty(price, cash_flow, periods, per):
    def present_value(rate, cash_flow, periods, per):
        discount_factor = (1 + rate/per)**(-periods*per)
        return np.dot(cash_flow, discount_factor)
    def f(x):
        return present_value(rate=x, cash_flow=cash_flow, periods=periods, per=per) - price
    root = newton(f, 0, disp=False)
    return root

at_return = aty(price = buy_price, cash_flow=at_cf, periods=np.arange(1, n_sell+1), per=2)
print(f'After-Tax Rate of Return = {at_return:0.3%} (s.a.)')

Holding Period Return 6.357% (s.a.), 2 years
Constant Yield Price (5.174%) = 63.145, year 1
Taxable Interest Income 3.145 Income Tax = 0.786
Constant Yield Price (5.174%) = 66.454, year 2
Taxable Interest Income 3.309 Income Tax = 0.827
Capital Gains Tax = 0.232 on Capital Gain = 1.546
After-Tax Rate of Return = 4.912% (s.a.)


### Municipal Bonds

Municipal bonds (munis) issued by state and local governments in the U.S. are **exempt from federal taxation**. Additionally, if the investor residers in the issuer's locality, the investor may also be exempt from state and local income taxes.

Investors often evaluate a municipal bond based on its **equivalent taxable yield** (ETY) statistic. The intent is to be able to compare directly the yield on a tax‐exempt bond to otherwise comparable fully taxable corporate offerings. This comparison is not as easy as it might sound because the bond rating agencies have different criteria for each bond type—that is, a double A‐rated muni does not necessarily have the same projected probability of default (and recovery rate) as a double A‐rated corporate bond.

The commonly used ETY statistic, which we can call the *street* version because it is widely used in practice, is the *after‐tax yield divided by one minus the assumed ordinary income tax rate*. Note that the after‐tax yield on the muni is not just its quoted, or pretax, yield to maturity. When a muni is purchased at a premium or discount and either held to maturity or sold prior to maturity, there still are federal tax implications, it generates taxable capital gains and losses for de minimis OID bonds or ordinary income tax for OID bonds on the discount portion.

These taxes are not economically justified. In principle, the “gain” from buying at a market discount is just deferred interest income and should be tax exempt similar to the coupon payments. Two bonds issued by the same state government, maturing on the same date with the same credit risk should have the same after-tax yield to maturity despite their coupon rates (differing due to liquidity, perhaps). However, the current tax regulationn penalizes the lower coupon discount bond because its coupon deficiency is classified as gains and is taxed as ordinary income—not as tax-exempt interest income.

Equivalent taxable yield (ETY) - Street Version:
$$ETY = \frac{\text{After-Tax YTM}}{(1 - \text{Ordinary Income Tax Rate})}$$

Equivalent taxable yield (ETY) - Theoretical Version:
$$PV=\left(\frac{PMT}{1-\text{Oridnary Income Tax Rate}}\right)\left(\frac{1-(1+ETY)^{-N}}{ETY}\right)+\frac{FV}{(1+ETY)^N}$$

The second, theoretical version, solve for the internal rate of return on a taxable offering that generates the same after‐tax cash flows as the muni. Note that the redemption amount does not need to be adjusted because both the taxable corporate and the “tax‐exempt” muni face capital gains taxation on the de minimis OID or ordinary income tax on market discount bonds.

The latter function (or the expanded version of the closed-form function) allows for more flexibility in including **term structure of tax rates** over time as the investor's ordinary tax rate increases. To incorporate this assumption into the street ETY calculation, one could input some form of weighted average of the tax rates.

In [75]:
muni_n = 4
muni_face = 100
muni1_cpn = 0.04
muni1_price = 99.342 # qualifies for de minimis OID rule as discount < (4 x 0.25)
muni2_cpn = 0.01
muni2_price = 88.499 # OID classification 

income_tax = 0.25
capital_tax = 0.15

muni1_cf = np.full(muni_n, muni1_cpn*100)
muni1_cf[-1] += muni_face - (muni_face - muni1_price)*capital_tax # discount taxed as capital gains (de minimis OID)

muni2_cf = np.full(muni_n, muni2_cpn*100)
muni2_cf[-1] += muni_face - (muni_face - muni2_price)*income_tax # discount taxes as ordinary interest income

def aty(price, cash_flow, periods, per):
    def present_value(rate, cash_flow, periods, per):
        discount_factor = (1 + rate/per)**(-periods*per)
        return np.dot(cash_flow, discount_factor)
    def f(x):
        return present_value(rate=x, cash_flow=cash_flow, periods=periods, per=per) - price
    root = newton(f, 0, disp=False)
    return root

muni1_at_return = aty(price = muni1_price, cash_flow=muni1_cf, periods=np.arange(1, muni_n+1), per=1)
street_ety_muni1 = muni1_at_return / (1 - income_tax)
print(f'Muni 1: After-Tax Rate of Return = {muni1_at_return:0.3%} (annually), Equiv. Taxable Yield (ETY) = {street_ety_muni1:0.3%}')

muni2_at_return = aty(price = muni2_price, cash_flow=muni2_cf, periods=np.arange(1, muni_n+1), per=1)
street_ety_muni2 = muni2_at_return / (1 - income_tax)
print(f'Muni 2: After-Tax Rate of Return = {muni2_at_return:0.3%} (annually), Equiv. Taxable Yield (ETY) = {street_ety_muni2:0.3%}')

Muni 1: After-Tax Rate of Return = 4.159% (annually), Equiv. Taxable Yield (ETY) = 5.545%
Muni 2: After-Tax Rate of Return = 3.444% (annually), Equiv. Taxable Yield (ETY) = 4.592%


In [76]:
muni1_cf = np.full(muni_n, muni1_cpn*100)
muni1_cf[-1] += muni_face

muni2_cf = np.full(muni_n, muni2_cpn*100)
muni2_cf[-1] += muni_face

muni1_at_return = aty(price = muni1_price, cash_flow=muni1_cf, periods=np.arange(1, muni_n+1), per=1)
print(f'Muni 1: Pre-Tax Rate of Return = {muni1_at_return:0.3%} (annually)')

muni2_at_return = aty(price = muni2_price, cash_flow=muni2_cf, periods=np.arange(1, muni_n+1), per=1)
print(f'Muni 2: Pre-Tax Rate of Return = {muni2_at_return:0.3%} (annually)')

Muni 1: Pre-Tax Rate of Return = 4.182% (annually)
Muni 2: Pre-Tax Rate of Return = 4.182% (annually)


In [78]:
def aty(price, at_pmt, years, face):
    def present_value(rate, years, pmt, fv):
        old_settings = np.seterr(invalid='ignore')
        pvPMT = np.where(rate == 0, years, np.divide(1 - (1+rate)**-years, rate)) * pmt
        np.seterr(**old_settings)
        pvFV = fv / (1 + rate)**years
        return pvPMT + pvFV
    def f(x):
        return present_value(rate=x, years=years, pmt=at_pmt, fv=face) - price
    root = newton(f, 0, disp=False)
    return root

muni1_at_return = aty(price=muni1_price, at_pmt=(muni1_cpn*100)/(1-income_tax), years=muni_n, face=muni_face)
print(f'Muni 1: After-Tax Rate of Return = {muni1_at_return:0.3%} (annually)')

muni2_at_return = aty(price=muni2_price, at_pmt=(muni2_cpn*100)/(1-income_tax), years=muni_n, face=muni_face)
print(f'Muni 2: After-Tax Rate of Return = {muni2_at_return:0.3%} (annually)')

Muni 1: After-Tax Rate of Return = 5.521% (annually)
Muni 2: After-Tax Rate of Return = 4.542% (annually)
